<a href="https://www.kaggle.com/code/binfeng2021/question-answering-with-chinese-history-book?scriptVersionId=207856092" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Main Goal

* Process a pdf document of a Chinese hitory book, and use it as reference to answer fact-based questions related to Chinese history.
* Embed the texts from the book and find the most similar text chunk as context to answer provided questions. 
* Build a questions answering bot that can process the information and find the most likely answer based on contexts.

# Data 
[A History of CHINA by Morris Rossabi](https://arxiujosepserradell.cat/wp-content/uploads/2021/12/A-History-of-China-by-Morris-Rossabi-z-lib.org_.pdf)

## Install and import needed libraries

In [1]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 64.4 MB/s eta 0:00:00:00:0100:01


In [2]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 3.6 MB/s eta 0:00:0000:010:01


In [3]:
import pymupdf
import re
import json
import pandas as pd
import math

from sentence_transformers import SentenceTransformer, util

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

from collections import Counter

from fuzzywuzzy import fuzz

import nltk
nltk.download('punkt')  # Download the sentence tokenizer

from nltk.tokenize import sent_tokenize
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
def get_most_popular_item(l):
    counter = Counter(l)
    most_common = counter.most_common(1)
    value, _ = most_common[0]
    return value

In [5]:
def extract_pdf_text_with_metadata(page_num, doc):
    data = {'text':[], 'bbox':[], 'page':[], 'font':[], 'size':[]}
    current_part = ""
    min_pos = math.inf
    page = doc.load_page(page_num)
    
    # extract text blocks with positional info
    blocks = page.get_text("dict")["blocks"]
    
    for block in blocks:
        if "lines" not in block:
            continue
        
        bbox = block['bbox']
        block_texts = ""
        font_list = []
        size_list = []
        
        for line in block["lines"]:
            line_texts = ""
            for span in line["spans"]:
                line_texts += span["text"].strip()
                font_list.append(span["font"])
                size_list.append(span["size"])
                
            if block_texts == "":
                block_texts = line_texts
            else:
                block_texts = block_texts + " " + line_texts

        # store text blocks with metadata
        # first, choose the most popular font and size of the block to store
        
        data['font'].append(get_most_popular_item(font_list))
        data['size'].append(get_most_popular_item(size_list))
        data['text'].append(block_texts)
        data['bbox'].append(bbox)
        data['page'].append(page_num)
    data = pd.DataFrame(data)
    return data

In [6]:
def dict_to_df(book_dict):
    rows = []
    for part, part_info in book_dict.items():
        part_name = part_info["name"]
        for chapter, chapter_info in part_info["chapters"].items():
            chapter_name = chapter_info["name"]
            for section, pages in chapter_info["sections"].items():
                rows.append([part_name, chapter_name, section, pages])
    
    # Create a DataFrame from the list of rows
    df = pd.DataFrame(rows, columns=["Part", "Chapter", "Section", "Pages"])
    return df

In [7]:
def find_most_similar(text, df, model, n = 5):
    # Encode the user query
    query_embedding = model.encode(text)
    
    # Calculate cosine similarities
    similarities = cosine_similarity([query_embedding], df['embeddings'].tolist())
    
    # Find the index of the most similar text
    # most_similar_idx = similarities.argmax()
    top_n_indices = np.argsort(similarities[0])[-n:][::-1]
    
    # Return the relevant info (text, page number, part, and chapter)
    relevant_info = df_process_pdf.iloc[top_n_indices]
    relevant_info.reset_index(drop = True, inplace = True)
    relevant_info['relevant_rank'] = relevant_info.index + 1
    return relevant_info

In [53]:
def get_context(context_info, context_window = 10):
    context_list = []
    relevant_rank_list = []
    for _, row in context_info.iterrows():
        sen_num = row['sen_num']
        part = row['Part']
        chapter = row['Chapter']
        section = row['Section']
        relevant_rank = row['relevant_rank']
        sentences_list = df_process_pdf[(df_process_pdf['Part']==part)
        &(df_process_pdf['Chapter']==chapter)
        &(df_process_pdf['Section']==section)
        &(df_process_pdf['sen_num']>=sen_num - context_window)
        &(df_process_pdf['sen_num']<=sen_num + context_window)].sentences.tolist()
        context = ' '.join(sentences_list)
        context_list.append(context)
        relevant_rank_list.append(relevant_rank)
    return context_list, relevant_rank_list

In [9]:
def generate_similarity_score(row):
    similarity = util.cos_sim(row['gt_embeddings'], row['predict_embeddings'])
    return similarity.item()

## Process the PDF documents

In [10]:
doc = pymupdf.open('/kaggle/input/a-history-of-china-morris-rossabi/A-History-of-China-by-Morris-Rossabi-z-lib.org_.pdf')

df_full_doc = []
for page_n in range(doc.page_count):
    processed_df = extract_pdf_text_with_metadata(page_n, doc)
    processed_df['page_num'] = page_n + 1
    df_full_doc.append(processed_df)

df_full_doc = pd.concat(df_full_doc)

df_full_doc['y0'] = df_full_doc['bbox'].apply(lambda x: x[1])
df_full_doc = df_full_doc.groupby('page_num', as_index=False).apply(lambda x: x.sort_values('y0').reset_index(drop = True).reset_index())
df_full_doc['block_num'] = df_full_doc['index'] + 1

df_full_doc = df_full_doc[(df_full_doc['font'].isin(['PlantinStd', 'PlantinStd-Italic-SC700']))&(df_full_doc['size'].round(2)>=10)&(df_full_doc['page_num']>=31)&(df_full_doc['page_num']<441)]

df_full_doc.loc[(df_full_doc['font']=='PlantinStd-Italic-SC700')&(df_full_doc['size'].round(2)==19.47), 'text_function'] = 'section title'

df_full_doc.loc[(df_full_doc['font']=='PlantinStd')&(df_full_doc['text_function'].isnull())&(~df_full_doc['text'].str.startswith('Figure'))&(~df_full_doc['text'].str.startswith('Map')), 'text_function'] = 'main body'

df_in = df_full_doc[df_full_doc['text_function'].isin(['main body', 'section title'])][['page_num', 'block_num', 'text', 'text_function']].drop_duplicates().sort_values(['page_num', 'block_num']).reset_index(drop = True)

In [11]:
section_list = df_in[df_in['text_function']=='section title'].index.tolist() + [math.inf]

In [12]:
dict_ref = {'section':[], 'text':[]}
for (start, end) in zip(section_list, section_list[1:]):
    df = df_in[(df_in.index >start)&(df_in.index <end)]
    section = df_in[df_in.index==start].text.values[0]
    dict_ref['section'].append(section)
    text_all = " ".join(df['text'].unique().tolist())
    dict_ref['text'].append(text_all)

dict_ref = pd.DataFrame(dict_ref)

# Remove further reading and notes sections
dict_ref = dict_ref[~dict_ref['section'].isin(['FURTHERREADING', 'NOTES'])]

In [13]:
with open('/kaggle/input/a-history-of-china-morris-rossabi/table of contents.txt') as file:
    content = file.read()

In [14]:
# Convert part of the dictionary to DataFrame
df_structure = dict_to_df(json.loads(content))

# note that the page number we will be using is the page of the pdf document, not the book. There are a difference of 28 between book page and pdf page
df_structure['book_page'] = df_structure['Pages'].str.extract('(\d+)')
df_structure['page_start'] = df_structure['book_page'].astype(float) + 28 # offset the page differences, so we will talk about pdf pages going forward
df_structure['page_end'] = df_structure['page_start'].shift(-1)

df_structure.loc[df_structure['page_end'].isnull(), 'page_end'] = df_structure.loc[df_structure['page_end'].isnull(), 'page_start']

In [15]:
extracted_sections = dict_ref['section'].unique().tolist()
original_sections = df_structure['Section'].unique().tolist()

section_match = {'section':[], 'Section':[], 'ratio':[]}
for l1 in extracted_sections:
    l1_ = l1.replace(' ', '')
    l1_ = l1_.replace(r'[^\w\s]', '')
    for l2 in original_sections:
        l2_ = l2.upper()
        l2_ = l2_.replace(' ', '')
        l2_ = l2_.replace(r'[^\w\s]', '')
        ratio = fuzz.ratio(l1_, l2_)
        section_match['section'].append(l1)
        section_match['Section'].append(l2)
        section_match['ratio'].append(ratio)

section_match = pd.DataFrame(section_match).sort_values('ratio', ascending=False).groupby(['section'], as_index=False).head(1).drop('ratio', axis = 1)

In [16]:
df_process_pdf = df_structure.merge(section_match).merge(dict_ref).drop('section', axis = 1)

In [17]:
df_process_pdf['sentences'] = df_process_pdf['text'].apply(lambda x: sent_tokenize(x))

In [18]:
df_process_pdf = df_process_pdf.explode('sentences').reset_index(drop = True)

In [19]:
# build text embeddings
embeddings_model = SentenceTransformer("all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
df_process_pdf['embeddings'] = df_process_pdf['sentences'].apply(lambda x: embeddings_model.encode(x, show_progress_bar = False))

In [21]:
df_process_pdf = df_process_pdf.groupby(['Part', 'Chapter', 'Section'], as_index=False).apply(lambda x: x.reset_index(drop = True).reset_index())

df_process_pdf.rename(columns = {'index':'sen_num'}, inplace = True)

In [35]:
df_process_pdf.head()

sen_num                      Part  \
0 0        0  China Among 'Barbarians'   
  1        1  China Among 'Barbarians'   
  2        2  China Among 'Barbarians'   
  3        3  China Among 'Barbarians'   
  4        4  China Among 'Barbarians'   

                                               Chapter                Section  \
0 0  Chaos and Religious and Political Responses, 2...  Buddhism Enters China   
  1  Chaos and Religious and Political Responses, 2...  Buddhism Enters China   
  2  Chaos and Religious and Political Responses, 2...  Buddhism Enters China   
  3  Chaos and Religious and Political Responses, 2...  Buddhism Enters China   
  4  Chaos and Religious and Political Responses, 2...  Buddhism Enters China   

         Pages book_page  page_start  page_end  \
0 0  Pages 110       110       138.0     144.0   
  1  Pages 110       110       138.0     144.0   
  2  Pages 110       110       138.0     144.0   
  3  Pages 110       110       138.0     144.0   
  4  Pages 110       110       138.0     144.0   

                                                  text  \
0 0  Buddhism is linked with the Indian prince Sidd...   
  1  Buddhism is linked with the Indian prince Sidd...   
  2  Buddhism is linked with the Indian prince Sidd...   
  3  Buddhism is linked with the Indian prince Sidd...   
  4  Buddhism is linked with the Indian prince Sidd...   

                                             sentences  \
0 0  Buddhism is linked with the Indian prince Sidd...   
  1  Because descriptions of his life evolved centu...   
  2  Such writings are not reliable accounts, as th...   
  3  Actual events are so interlaced with myths and...   
  4  Yet  consideration of the mythical accounts co...   

                                            embeddings  
0 0  [0.038585927, 0.020595705, -0.035208672, 0.007...  
  1  [0.021002242, 0.016055271, -0.04977115, 0.0365...  
  2  [0.086978056, 0.028460192, -0.01349521, 0.0572...  
  3  [0.06389172, 0.051194068, -0.015496978, 0.0812...  
  4  [0.030101553, 0.05770132, -0.032306902, 0.0712...

## Build QA bot to answer questions

In [22]:
qa_pipeline = pipeline("question-answering", model = "distilbert-base-uncased-distilled-squad")

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [23]:
df_questions = pd.read_excel("/kaggle/input/a-history-of-china-morris-rossabi/QA_dataset.xlsx", sheet_name = 'Sheet1')

In [54]:
df_results = []
c = 0
for row, df_row in df_questions.iterrows():
    question = df_row.Question
    context_info = find_most_similar(question, df_process_pdf, embeddings_model)
    context_list, relevant_rank_list = get_context(context_info)
    # Get the answer
    for context, relevant_rank in zip(context_list, relevant_rank_list):
        result = qa_pipeline(question=question, context=context)
        df_row['generated_answer'] = result['answer']
        df_row['score'] = result['score']
        df_row['relevant_rank'] = relevant_rank
        df_results.append(pd.DataFrame(df_row).T)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [55]:
df_results = pd.concat(df_results)

## Evaluate the Results Generated From the Model

There are two approaches that I created to generate the answers:

1. Use the texts with highest rank as the context to feed to the QA model, and generate the answer;
2. Use the top 5 highest rank texts as the contexts, and generate the answer for each context. Suming the scores up for each generarted answer and provide the answer with highest total scoring as the final answer.

In [56]:
# first method, we will only use the answer generated from the highest rank context
df_results1 = df_results[df_results['relevant_rank']==1]

df_results1.sort_values('score', ascending=False).head(10)

,Question,Answer,generated_answer,score,relevant_rank
42,"Who was the founder of the Republic of China, ...",Sun Yat-sen,Sun Yat-sen,0.997808,1
7,"Who was known as the ""Yellow Emperor"" and was ...",Huangdi,Huangdi,0.99766,1
35,Which Qing dynasty painter is known for his ex...,Zhu Da (Bada Shanren),Dong Yuan,0.99558,1
49,Which city served as the primary capital of th...,Hangzhou,Hangzhou,0.995075,1
20,Which city was the prosperous capital during t...,Hangzhou,Hangzhou,0.993635,1
43,Who was the influential female poet and court ...,Li Qingzhao,Li Shangyin,0.992143,1
8,Which Tang dynasty poet was known for his empa...,Bo Juyi,Li Shangyin,0.990271,1
45,Where is the site of one of the earliest civil...,"Banpo, near Xi'an in Shaanxi province",Gansu,0.987581,1
70,Who was the famous Tang poet known for his nat...,Li Bai,Li Shangyin,0.98683,1
2,Which two legendary rulers were key models of ...,Yao and Shun,Yao and Shun,0.983648,1


In [57]:
# second method, we will calculate the total score for each generated answers from the top 5 contexts, then choose the answer with highest score
df_results2 = df_results.groupby(['Question', 'Answer', 'generated_answer'], as_index=False)['score'].sum()
df_results2 = df_results2.sort_values('score', ascending=False).groupby(['Question', 'Answer'], as_index=False).head(1)
df_results2.sort_values('score', ascending=False).head(10)

,Question,Answer,generated_answer,score
442,Who was the Chinese philosopher known for his ...,Han Fei,Guan Zhong,3.645005
275,Which city was the prosperous capital during t...,Hangzhou,Hangzhou,2.977944
164,Where is the site of one of the earliest civil...,"Banpo, near Xi'an in Shaanxi province",Gansu,2.921512
384,"Which site, discovered in the 20th century, is...",Erlitou,Erlitou,2.909402
8,What ancient trade route connected China with ...,Silk Road,Silk Roads,2.369487
308,Which early Chinese philosopher's teachings sh...,Confucius,Confucius,2.08423
500,"Who was the founder of the Republic of China, ...",Sun Yat-sen,Sun Yat-sen,1.995508
268,Which city served as the primary capital of th...,Hangzhou,Hangzhou,1.98342
220,Which Qing dynasty painter is known for his ex...,Zhu Da (Bada Shanren),Dong Yuan,1.976473
496,"Who was the founder of the Han dynasty, rising...","Liu Bang, later Emperor Gaozu",Xiang Yu,1.976012


### Use semantic similarity to evaluate the generated answers against the provided answers

In [58]:
# using sentence transformer to embed the generated answers and the ground truth, so we can calculate the semantic similarity
df_results1['gt_embeddings'] = df_results1['Answer'].apply(lambda x: embeddings_model.encode(x, show_progress_bar = False))
df_results1['predict_embeddings'] = df_results1['generated_answer'].apply(lambda x: embeddings_model.encode(x, show_progress_bar = False))
df_results1['similarity'] = df_results1.apply(generate_similarity_score, axis = 1)

# calculate similarity for results 2
df_results2['gt_embeddings'] = df_results2['Answer'].apply(lambda x: embeddings_model.encode(x, show_progress_bar = False))
df_results2['predict_embeddings'] = df_results2['generated_answer'].apply(lambda x: embeddings_model.encode(x, show_progress_bar = False))
df_results2['similarity'] = df_results2.apply(generate_similarity_score, axis = 1)

In [59]:
df_results1[df_results1['similarity']>=0.8].shape

(25, 8)

In [61]:
df_results2[df_results2['similarity']>=0.8].shape

(26, 7)

# Future Work

* Improve the accuracy of the context extraction. Currently I comparied the similarity of provided question and each senteces of the book to find the most similar sentence. Then I expand to a block of texts based on window size. However, based on how the question is phrased, sometimes it could lead to the incorrect contexts.

* The question answering model is pre-trained model, the results generated might not be the most optimal yet. However, the proper dataset is lacking currently to finetune the model. 